**Cleaning For Data**
**Horizontally Add FIFA Data year by year**

In [10]:
import pandas as pd
import openpyxl
RAW_DATA_DIR = '../data/raw'

In [19]:
df_fb = pd.read_csv(f'{RAW_DATA_DIR}/fbref_multi_season_performance_data.csv', dtype=str)
print(f"Columns in dataframe: {df_fb.columns.tolist()[:10]}...")
df_fb.head(5)

Columns in dataframe: ['league', 'season', 'team', 'player', 'nation', 'pos', 'age', 'born', 'Playing Time_MP', 'Playing Time_Starts']...


,league,season,team,player,nation,pos,age,born,Playing Time_MP,Playing Time_Starts,...,Per 90 Minutes_Gls,Per 90 Minutes_Ast,Per 90 Minutes_G+A,Per 90 Minutes_G-PK,Per 90 Minutes_G+A-PK,Per 90 Minutes_xG,Per 90 Minutes_xAG,Per 90 Minutes_xG+xAG,Per 90 Minutes_npxG,Per 90 Minutes_npxG+xAG
0,BRA-Serie A,1415,Ath Paranaense,Bady,BRA,MF,24,1989,26,23,...,0.16,0.16,0.32,0.16,0.32,NaN,NaN,NaN,NaN,NaN
1,BRA-Serie A,1415,Ath Paranaense,Bruno Furlan,BRA,FW,21,1992,1,0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,BRA-Serie A,1415,Ath Paranaense,Bruno Mendes,BRA,FW,19,1994,3,0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,BRA-Serie A,1415,Ath Paranaense,Carlos Alberto,BRA,MF,25,1988,1,0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,BRA-Serie A,1415,Ath Paranaense,Cleberson,BRA,DF,21,1992,34,34,...,0.15,0.0,0.15,0.15,0.15,NaN,NaN,NaN,NaN,NaN


In [12]:
# Remove players in df_fb with single names (likely data quality issues)
print(len(df_fb))
df_fb = df_fb[df_fb['player'].str.strip().str.count(' ') > 0]
print(len(df_fb))

62594
57008


In [13]:
# Create name key (unique per player per season)
# Example: Carlos Albert -> Carlos Alberto1988BRA
df_fb['name_key'] = (
    df_fb['player'] +
    df_fb['born'] + 
    df_fb['nation'] 
)
df_fb[['name_key', 'player', 'nation', 'born']].head(5)

,name_key,player,nation,born
1,Bruno Furlan1992BRA,Bruno Furlan,BRA,1992
2,Bruno Mendes1994BRA,Bruno Mendes,BRA,1994
3,Carlos Alberto1988BRA,Carlos Alberto,BRA,1988
9,Douglas Coutinho1994BRA,Douglas Coutinho,BRA,1994
11,Felipe Silva1990BRA,Felipe Silva,BRA,1990


In [14]:
# Need to remove rows where players played for two teams in one season, make row w more minutes the sum of stats, remove other row
# Then the sheet should be ready to merge with fifa data on name_key

In [23]:
df_fifa = pd.read_csv(f'{RAW_DATA_DIR}/fifa_data.csv', dtype=str)
df_fifa.head(10)

,player_id,player_url,fifa_version,fifa_update,update_as_of,short_name,long_name,player_positions,overall,potential,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,231747,/player/231747/kylian-mbappe/240002,24.0,2.0,2023-09-22,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,94,...,63+3,63+3,63+3,68+3,63+3,54+3,54+3,54+3,63+3,18+3
1,239085,/player/239085/erling-haaland/240002,24.0,2.0,2023-09-22,E. Haaland,Erling Braut Haaland,ST,91,94,...,63+3,63+3,63+3,62+3,60+3,62+3,62+3,62+3,60+3,19+3
2,192985,/player/192985/kevin-de-bruyne/240002,24.0,2.0,2023-09-22,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,...,80+3,80+3,80+3,79+3,75+3,70+3,70+3,70+3,75+3,21+3
3,158023,/player/158023/lionel-messi/240002,24.0,2.0,2023-09-22,L. Messi,Lionel Andrés Messi Cuccittini,"CF, CAM",90,90,...,63+3,63+3,63+3,64+3,59+3,49+3,49+3,49+3,59+3,19+3
4,165153,/player/165153/karim-benzema/240002,24.0,2.0,2023-09-22,K. Benzema,Karim Benzema,"CF, ST",90,90,...,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3
5,188545,/player/188545/robert-lewandowski/240002,24.0,2.0,2023-09-22,R. Lewandowski,Robert Lewandowski,ST,90,90,...,66+3,66+3,66+3,65+3,61+3,61+3,61+3,61+3,61+3,19+3
6,192119,/player/192119/thibaut-courtois/240002,24.0,2.0,2023-09-22,T. Courtois,Thibaut Nicolas Marc Courtois,GK,90,90,...,31+3,31+3,31+3,29+3,29+3,29+3,29+3,29+3,29+3,88+2
7,202126,/player/202126/harry-kane/240002,24.0,2.0,2023-09-22,H. Kane,Harry Kane,ST,90,90,...,69+3,69+3,69+3,67+3,64+3,63+3,63+3,63+3,64+3,20+3
8,238794,/player/238794/vinicius-jose-de-oliveira-junio...,24.0,2.0,2023-09-22,Vini Jr.,Vinícius José Paixão de Oliveira Júnior,LW,89,94,...,58+3,58+3,58+3,62+3,57+3,45+3,45+3,45+3,57+3,16+3
9,212831,/player/212831/alisson-ramses-becker/240002,24.0,2.0,2023-09-22,Alisson,Alisson Ramsés Becker,GK,89,90,...,39+3,39+3,39+3,34+3,33+3,33+3,33+3,33+3,33+3,87+3


In [16]:
# TODO: Match fb name key -> use short name last name and mathch the 'X.' in short name to the corresponding name in long_name
# TODO: Create season column based on update_as_of year (simple if statements)
# TODO: Remove repeat columns
# TODO: Remove any players in fifa data not in fb data
# Then should be ready for horizontal concat/merge

In [ ]:
#Handle Cases where you can -> otherwise use short name
# Incoroporate position into name key to differentiate players with same name and birth year/nation

In [18]:
# Last name is everything after first space in short_name
df_fifa['last_name'] = df_fifa['short_name'].str.split(' ').str[1:].str.join(' ')

# Function to find corresponding piece of long_name that matches 'X.' in short_name
# Example: 'R. Lewandowski' -> 'Robert Lewandowski'
def match_long_name(row):
    short = str(row['short_name'])
    long = str(row['long_name'])
    
    # Get the first letter from short_name (the 'R' in 'R. Lewandowski')
    first_letter = short.split(' ')[0].replace('.', '').strip().upper()
    
    # Split long_name into words
    long_words = long.split(' ')
    
    # Find the first word in long_name that starts with the same letter
    for word in long_words:
        if word and word[0].upper() == first_letter:
            # Reconstruct: first_word + last_name from short_name
            last_name = ' '.join(short.split(' ')[1:])
            return f"{word} {last_name}"
    
    # Fallback: if no match found, return the original long_name
    return long

df_fifa['matched_long_name'] = df_fifa.apply(match_long_name, axis=1)
df_fifa[['short_name', 'long_name', 'matched_long_name']].head(10)

,short_name,long_name,matched_long_name
0,K. Mbappé,Kylian Mbappé Lottin,Kylian Mbappé
1,E. Haaland,Erling Braut Haaland,Erling Haaland
2,K. De Bruyne,Kevin De Bruyne,Kevin De Bruyne
3,L. Messi,Lionel Andrés Messi Cuccittini,Lionel Messi
4,K. Benzema,Karim Benzema,Karim Benzema
5,R. Lewandowski,Robert Lewandowski,Robert Lewandowski
6,T. Courtois,Thibaut Nicolas Marc Courtois,Thibaut Courtois
7,H. Kane,Harry Kane,Harry Kane
8,Vini Jr.,Vinícius José Paixão de Oliveira Júnior,Vinícius José Paixão de Oliveira Júnior
9,Alisson,Alisson Ramsés Becker,Alisson Ramsés Becker
